In [1]:
import pinocchio as pin
import os
from pinocchio.visualize import MeshcatVisualizer
import numpy as np
from numpy.linalg import solve,norm
# from robot_descriptions.loaders.pinocchio import load_robot_description
# import gepetto.corbaserver

In [2]:
urdf_filename = "/home/adi/hum_rob_ws/src/six_dof/urdf/6dof_from_hip.urdf"
mesh_dir = "/home/adi/hum_rob_ws/src/six_dof/meshes"

ros_package_path = "/home/adi/hum_rob_ws/src"  # Replace with your path
os.environ["ROS_PACKAGE_PATH"] = ros_package_path

In [3]:
# building model from URDF, model is an object that includes kinematic and ineratia params
model = pin.buildModelFromUrdf(urdf_filename)

visual_model = pin.buildGeomFromUrdf(
    model, urdf_filename, pin.GeometryType.VISUAL, package_dirs=mesh_dir
)  # very important

collision_model = pin.buildGeomFromUrdf(
    model, urdf_filename, pin.GeometryType.COLLISION, package_dirs=mesh_dir
)

geom_data = pin.GeometryData(collision_model)


# data is an object that holds values that are a result of computation
data = model.createData()

# print('standard model: dim=' + str(len(model.joints)))
# for jn in model.joints:
#     print(jn)
# print('-' * 30)


# # Create a list of joints to lock
# jointsToLock = ['right_foot_shin', 'right_shin_thigh', 'right_thigh_body']
 
# # Get the ID of all existing joints
# jointsToLockIDs = []
# for jn in jointsToLock:
#     if model.existJointName(jn):
#         jointsToLockIDs.append(model.getJointId(jn))
#     else:
#         print('Warning: joint ' + str(jn) + ' does not belong to the model!')

# print(jointsToLockIDs)
 
# geom_models = [visual_model, collision_model]
# model_reduced, geometric_models_reduced = pin.buildReducedModel(
#     model,
#     list_of_geom_models=geom_models,
#     list_of_joints_to_lock=jointsToLockIDs,
#     reference_configuration=pin.neutral(model))
# # geometric_models_reduced is a list, ordered as the passed variable "geom_models" so:
# visual_model_reduced, collision_model_reduced = geometric_models_reduced[
#     0], geometric_models_reduced[1]

In [4]:
visualizer = MeshcatVisualizer(model, collision_model, visual_model)


visualizer.initViewer()
visualizer.loadViewerModel()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7013/static/


In [5]:
q = pin.neutral(model)
print(q)
eps = 1e-3
DT = 1e-1
damp = 1e-1
IT_MAX = 20000
JOINT_ID = 8


R_des = np.array([[1, 0, 0], [0, 0.000796327, -1], [0, 1, 0.000796327]])
#R_des = np.eye(3)

oMdes = pin.SE3(
    R_des, np.array([0.0235,-0.030,-0.265])
)  # think of oMdes as oTdes, the transfromation from desired to origin expressed in origin
i = 0

[0. 0. 0. 0. 0. 0. 0. 0.]


In [6]:
visualizer.display(q)
visualizer.viewer.jupyter_cell()

In [7]:
while True:
    pin.forwardKinematics(model, data, q)
    
    iMd = data.oMi[JOINT_ID].actInv(oMdes) #active inverse, chnages basis and moves the vector/point to the appropriate position in the new basis
    err = pin.log(iMd).vector  # in joint frame  The logarithm map (log()) is a mathematical operation that maps elements from the group SE(3) to its associated Lie algebra, se(3). This operation effectively converts the complex rotational and translational transformation into a simpler vector representation that captures the "difference" or "error" between two poses in SE(3).
    norm_err=norm(err)
    if norm_err < eps:
        success = True
        break
    if i >= IT_MAX:
        success = False
        break
    J = pin.computeJointJacobian(
        model, data, q, JOINT_ID
    )  # jacobian computed in e-e frame
    J = -np.dot(pin.Jlog6(iMd.inverse()), J)
    q_dot = -J.T.dot(solve(J.dot(J.T) + damp * np.eye(6), err))
    q = pin.integrate(model, q, q_dot * DT)
    
    #Collision check
    pin.updateGeometryPlacements(
        model, data, collision_model, geom_data, q
    )  # updates geom_data by reference

    # an active pair is a pair of bodies in a joint considered for collision
    collision_detected = pin.computeCollisions(
        model, data, collision_model, geom_data, q, True
    )  # this returns data,geom_data. polymorphic function

    if not collision_detected:
        for idx in range(model.nq):
            # Assuming model.lowerPositionLimit and model.upperPositionLimit store the limits
            q[idx] = max(
                model.lowerPositionLimit[idx],
                min(q[idx], model.upperPositionLimit[idx]),
            )
    
        print(q)
        visualizer.display(q)
    print("*************"+str(norm_err)+"***********")
    #print(i)
    i+=1
    #time.sleep(0.05)

if success:
    print("Convergence achieved!")
else:
    print(
        "\nWarning: the iterative algorithm has not reached convergence to the desired precision"
    )

print("\nresult: %s" % q.flatten().tolist())
print("\nfinal error: %s" % err.T)



[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  3.97324646e-05 -3.19856036e-03  1.10507740e-04  2.79912557e-03]
*************0.03586354093097808***********
[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  7.57835595e-05 -6.30039623e-03  2.27443797e-04  5.53102883e-03]
*************0.0351086569143353***********
[ 0.          0.          0.          0.          0.00010849 -0.00930905
  0.00034973  0.00819663]
*************0.034385342545569575***********
[ 0.          0.          0.          0.          0.00013815 -0.01222787
  0.00047642  0.0107969 ]
*************0.033691831187415595***********
[ 0.          0.          0.          0.          0.00016505 -0.01506006
  0.00060665  0.01333287]
*************0.03302654956196549***********
[ 0.          0.          0.          0.          0.00018943 -0.01780863
  0.00073966  0.01580564]
*************0.03238809051782688***********
[ 0.          0.          0.          0.          0.00021154 -0.02047648
  0.

In [8]:
pin.forwardKinematics(model,data,q)
 
# Print out the placement of each joint of the kinematic tree
for name, oMi in zip(model.names, data.oMi):
    print(("{:<24} : {: .3f} {: .3f} {: .3f}"
          .format( name, *oMi.translation.T.flat )))

print(data.oMi[8])

universe                 :  0.000  0.000  0.000
body_left_thigh_lateral  :  0.100  0.000  0.000
body_left_thigh          :  0.100  0.000  0.000
left_thigh_shin          :  0.142 -0.000 -0.160
left_shin_foot           :  0.123 -0.000 -0.290
right_body_thigh_lateral :  0.000  0.000  0.000
right_body_thigh         :  0.041  0.000  0.000
right_thigh_shin         :  0.044 -0.066 -0.146
right_shin_foot          :  0.024 -0.030 -0.271
  R =
           1 -2.17471e-05  3.97387e-10
 1.73178e-08  0.000778054           -1
 2.17471e-05            1  0.000778054
  p = 0.0235059 -0.030125 -0.270992



In [9]:
result_deg=[np.rad2deg(i) for i in q]
print(result_deg)

[0.0, 0.0, 0.0, 0.0, 0.0012460190527387829, -24.16332510573007, 40.049755337167234, -15.885383261996886]
